In [2]:
!pip install librosa
!pip install noisereduce
!pip install scipy

In [3]:
import librosa
import librosa.display
import numpy as np
import noisereduce as nr
import scipy.signal as signal
import soundfile as sf

In [4]:
# Load the audio file
file_path = "/content/R1023_20231115_060003.wav"  # Change this to your actual file path
y, sr = librosa.load(file_path, sr=None)

# Apply a Band-Pass Filter (Keep bird frequencies: ~1kHz to 8kHz)
lowcut = 1000  # 1 kHz
highcut = 8000  # 8 kHz

def bandpass_filter(data, sr, lowcut, highcut, order=6):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype="band")
    return signal.filtfilt(b, a, data)

filtered_audio = bandpass_filter(y, sr, lowcut, highcut)

# Noise Reduction using noisereduce
reduced_noise = nr.reduce_noise(y=filtered_audio, sr=sr, prop_decrease=0.8)

# Save the cleaned bird sound
output_file = "bird_sounds_extracted.wav"
sf.write(output_file, reduced_noise, sr)

print(f"Processed audio saved as {output_file}")

Processed audio saved as bird_sounds_extracted.wav


In [5]:
print(sr)

32000


In [4]:
import librosa
import librosa.display
import numpy as np
import noisereduce as nr
import scipy.signal as signal
import soundfile as sf

# Load the audio file
file_path = "/content/R1023_20231115_060003.wav"  # Change this to your actual file path
y, sr = librosa.load(file_path, sr=None)

# Apply a Band-Pass Filter (Keep bird frequencies: ~1kHz to 8kHz)
lowcut = 1000  # 1 kHz
highcut = 8000  # 8 kHz

def bandpass_filter(data, sr, lowcut, highcut, order=6):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype="band")
    return signal.filtfilt(b, a, data)

filtered_audio = bandpass_filter(y, sr, lowcut, highcut)

# Stronger Noise Reduction using noisereduce
reduced_noise = nr.reduce_noise(y=filtered_audio, sr=sr, prop_decrease=0.9)

# Spectral Subtraction for further noise removal
def spectral_subtraction(audio, noise_factor=0.05):
    stft = librosa.stft(audio)
    magnitude, phase = np.abs(stft), np.angle(stft)
    noise_profile = np.mean(magnitude, axis=1, keepdims=True) * noise_factor
    enhanced_magnitude = np.maximum(magnitude - noise_profile, 0)
    return librosa.istft(enhanced_magnitude * np.exp(1j * phase))

cleaned_audio = spectral_subtraction(reduced_noise)

# Adaptive Gain Increase (Normalize & Amplify)
def adaptive_gain(audio, target_dB=-20):
    rms = np.sqrt(np.mean(audio ** 2))
    scalar = 10 ** ((target_dB - 20 * np.log10(rms)) / 20)
    return np.clip(audio * scalar, -1, 1)  # Prevent clipping

final_audio = adaptive_gain(cleaned_audio)

# Save the cleaned bird sound
output_file = "bird_sounds_extracted_enhanced.wav"
sf.write(output_file, final_audio, sr)

print(f"Processed audio saved as {output_file}")


Processed audio saved as bird_sounds_extracted_enhanced.wav


In [ ]:
import librosa
import librosa.display
import numpy as np
import noisereduce as nr
import scipy.signal as signal
import soundfile as sf

# Load the full 1-hour audio
file_path = "/content/R1023_20231115_060003.wav"
y, sr = librosa.load(file_path, sr=None, duration=None)  # Ensure full duration is loaded

# Bandpass Filter (Preserves More Sound)
lowcut = 1000
highcut = 8000

def bandpass_filter(data, sr, lowcut, highcut, order=3):  # Lower order to preserve sound
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype="band")
    return signal.filtfilt(b, a, data)

filtered_audio = bandpass_filter(y, sr, lowcut, highcut)

# Noise Reduction with Less Aggressive Reduction
reduced_noise = nr.reduce_noise(y=filtered_audio, sr=sr, prop_decrease=0.7)  # Less aggressive

# Spectral Subtraction (Avoid Over-Filtering)
def spectral_subtraction(audio, noise_factor=0.02):  # Reduce noise factor
    stft = librosa.stft(audio, n_fft=2048, hop_length=512)
    magnitude, phase = np.abs(stft), np.angle(stft)
    noise_profile = np.mean(magnitude, axis=1, keepdims=True) * noise_factor
    enhanced_magnitude = np.maximum(magnitude - noise_profile, 0)
    return librosa.istft(enhanced_magnitude * np.exp(1j * phase), hop_length=512)  # Match hop_length

cleaned_audio = spectral_subtraction(reduced_noise)

# Adaptive Gain (Preserve Full Audio Length)
def adaptive_gain(audio, target_dB=-25):  # Reduce gain to avoid clipping
    rms = np.sqrt(np.mean(audio ** 2))
    scalar = 10 ** ((target_dB - 20 * np.log10(rms)) / 20)
    return np.clip(audio * scalar, -1, 1)

final_audio = adaptive_gain(cleaned_audio)

# Ensure the Output is the Same Length as Input
if len(final_audio) < len(y):
    padding = np.zeros(len(y) - len(final_audio))
    final_audio = np.concatenate((final_audio, padding))

# Save the fixed file
output_file = "bird_sounds_extracted_full.wav"
sf.write(output_file, final_audio, sr)

print(f"Processed full 1-hour audio saved as {output_file}")
